In [2]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
ds = pd.read_csv('./mnist_train.csv').values
print (ds.shape)

(60000, 785)


In [17]:
X_train = ds[:10000, 1:].reshape((-1, 28, 28)) / 255.0
y_train = ds[:10000, 0]

X_test = ds[5000:5100, 1:].reshape((-1, 28, 28)) / 255.0
y_test = ds[5000:5100, 0]
# y_train = one_hot(y_train_labels, 10)

print (X_train.shape, y_train.shape)

(10000, 28, 28) (10000,)


In [18]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 12),
            nn.Tanh(),
            nn.Linear(12, 3),   # compress to 3 features which can be visualized in plt
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.Tanh(),
            nn.Linear(12, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 28*28),
            nn.Sigmoid(),       # compress to a range (0, 1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [19]:
AE = AutoEncoder()


In [20]:
print(AE)

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): Tanh()
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): Tanh()
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): Tanh()
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Sigmoid()
  )
)


In [21]:
def make_batch(train, labels, batch_size=10):
    start = 0
    stop = start + batch_size
    while start < train.shape[0]:
        yield Variable(torch.FloatTensor(train[start:stop]), requires_grad=True), Variable(torch.LongTensor(labels[start:stop]))
        start = stop
        stop = start + batch_size

In [22]:
optimizer = torch.optim.Adam(AE.parameters(), lr=0.0003) 

loss_func = nn.CrossEntropyLoss()       

In [23]:

for epoch in range(20):
    for step, (b_x, b_y) in enumerate(make_batch(X_train, y_train, 128)):   
        output = AE(b_x)              
        loss = loss_func(output, b_y)   
        optimizer.zero_grad()          
        loss.backward()               
        optimizer.step()            

        if step % 100 == 0:
            test_output = AE(torch.FloatTensor(X_test))
            outs = test_output.data.numpy().argmax(axis=1)
            acc = (outs == y_test).sum()*100.0 / test_output.shape[0]
            print('Epoch: ', epoch, '| Step: ', step, '| loss: ', acc)

RuntimeError: size mismatch, m1: [3584 x 28], m2: [784 x 128] at /pytorch/aten/src/TH/generic/THTensorMath.c:2033

# Linear regression and logistic regression with Regularization